# Traduction

**IMPORTANT** : Créez une ressource *Analytique de texte* ou *Text analytics* et renseignez `SERVEUR` avec votre point de terminaison (endpoint). Récupérez également la clé et renseignez là lors de l'exécution de la première cellule. Faites de même avec une ressource *Traducteur* ou *Translator* sur `TRAD_SERVEUR` en précisant la région dans `TRAD_REGION`.

In [35]:
from getpass import getpass

CLE = getpass("Clé d'analyse de texte")
SERVEUR = 'https://textanalysis00.cognitiveservices.azure.com'  # Votre point de terminaison
TRAD_CLE = getpass("Clé du traducteur")
TRAD_SERVEUR = 'https://api.cognitive.microsofttranslator.com/'  # Votre point de terminaison
TRAD_REGION = 'westeurope'

TEXTE1 = """
    There was a minute’s pause perhaps. The Psychologist seemed about to speak to me,
    but changed his mind. Then the Time Traveller put forth his finger towards the lever.
    'No,' he said suddenly. 'Lend me your hand.' And turning to the Psychologist, he took
    that individual's hand in his own and told him to put out his forefinger.
    So that it was the Psychologist himself who sent forth the model Time Machine
    on its interminable voyage.
"""
# H.G.Wells, The Time Machine, Chapter I

## Texte

In [36]:
import requests

reponse = requests.post(
    url = f'{TRAD_SERVEUR}/translate',
    headers={
      'Ocp-Apim-Subscription-Key': TRAD_CLE,
      'Ocp-Apim-Subscription-Region': TRAD_REGION
    },
    params = {
        'api-version': '3.0',
        'to': ['fr', 'es']
    },
    json=[{'text': TEXTE1}]
)
reponse.json()

[{'detectedLanguage': {'language': 'en', 'score': 1.0},
  'translations': [{'text': '\nIl y a peut-être eu une minute de pause. Le psychologue semblait sur le point de me parler,\n    mais a changé d’avis. Puis le voyageur temporel a avancé son doigt vers le levier.\n    «\xa0Non\xa0», dit-il soudainement. « Prête-moi ta main. » Et se tournant vers le psychologue, il a pris\n    Cet individu a mis la main dans la sienne et lui a dit de sortir son index.\n    De sorte que c’est le psychologue lui-même qui a envoyé le modèle de machine à remonter le temps\n    sur son voyage interminable.\n',
    'to': 'fr'},
   {'text': '\nHubo una pausa de un minuto tal vez. El psicólogo parecía a punto de hablarme,\n    pero cambió de opinión. Entonces el Viajero del Tiempo extendió su dedo hacia la palanca.\n    "No", dijo de repente. "Préstame tu mano". Y dirigiéndose al Psicólogo, tomó\n    La mano de ese individuo en la suya y le dijo que sacara su dedo índice.\n    De modo que fue el propio Psicó

## Traductions alternatives d'un mot

On réutilise l'analyse de texte pour isoler les mots importants.

In [37]:
import requests

reponse = requests.post(
  url = f'{SERVEUR}/text/analytics/v2.1/keyPhrases',
  headers = { 'Ocp-Apim-Subscription-Key': CLE },
  json = { 'documents': [{'id': '1', 'language': 'en', 'text': TEXTE1}] }
)
phrases = reponse.json()['documents'][0]['keyPhrases']
phrases

['Psychologist',
 'Time Traveller',
 "individual's hand",
 'model Time Machine',
 'interminable voyage',
 'minute’s pause',
 'lever',
 'mind',
 'forefinger']

On les cherche dans un dictionnaire.

In [38]:
reponse = requests.post(
    url = TRAD_SERVEUR + '/dictionary/lookup',
    headers = {
      'Ocp-Apim-Subscription-Key': TRAD_CLE,
      'Ocp-Apim-Subscription-Region': TRAD_REGION
    },
    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': 'fr'
    },
    json = [{ 'text': p } for p in phrases ]
)
[ { phrase['normalizedSource'] : 
    [ (trad['normalizedTarget'], trad['posTag']) 
      for trad in phrase['translations']
    ]
  } 
  for phrase in reponse.json()
]

[{'psychologist': [('psychologue', 'NOUN'), ('psy', 'NOUN')]},
 {'Time Traveller': []},
 {"individual's hand": []},
 {'model Time Machine': []},
 {'interminable voyage': []},
 {'minute’s pause': []},
 {'lever': [('levier', 'NOUN'), ('manette', 'NOUN')]},
 {'mind': [('esprit', 'NOUN'),
   ('mental', 'NOUN'),
   ('me dérange', 'VERB'),
   ('dérange', 'VERB'),
   ('tête', 'NOUN'),
   ('entendement', 'NOUN'),
   ('avis', 'NOUN'),
   ('pensée', 'NOUN')]},
 {'forefinger': [('index', 'NOUN')]}]